# Лабораторная работа №1

$\left\{ \omega_i \right\}$ --- равномерно распределённые на $ \left[ 0, 1 \right] $.

$N \left( 0, 1 \right) $ можно генерировать из последовательности $\left\{ \omega_i \right\} $:
$ \xi_1 = \sqrt{-2 \ln \omega_1} \cdot \sin \left( 2 \pi \cdot \omega_2 \right) ,
\xi_2 = \sqrt{-2 \ln \omega_1} \cdot \cos \left( 2 \pi \cdot \omega_2 \right) $ --- независимы.

Пусть наблюдается выборка $ \vec{X} = \left( X_1, \dotsc, X_n \right)$, где $X_i \sim N \left( 0, 1 \right) $.

Расчёты будем вести при $\gamma = 0.05$.
Это означает, что уровень доверия --- $0.95, z_{jamma} = 1.96$ (для нормального распределения).

$n = 100, 1000, 10000, 100000$.

In [1]:
from numpy.random import normal, uniform
from numpy import mean, std, log, exp, array, append, inf
from scipy.stats import norm, t, chi2
from scipy.integrate import quad
from math import sqrt
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [2]:
a = 0
sigma = 1
gamma = 0.05

## Задание 1
Определить скорость сходимости.
Действительно ли она $\frac{1}{\sqrt{n}}$?
Всегда ли доверительный интервал накрывает $0$?
Строим доверительный интервал в следующих случаях:

### а)
для математического ожидания $a$, когда дисперсия известна ($=1$).

Рассмотрим случай нормально распределённой генеральной совокупности, то есть допустим,
что $ \xi \sim N \left( a, \sigma^2 \right)$.

**Лемма 1** Введём обозначения для среднего арифметического
$$\hat{a}_n = \frac{1}{n} \sum \limits_{i=1}^n X_i$$
и для выборочной дисперсии
$$\hat{\sigma}_n^2 = \frac{1}{n} \sum \limits_{i=1}^n \left( X_i - \hat{a}_n \right)^2.$$
Тогда
$$\hat{a}_n \sim N \left( a, \frac{\sigma^2}{n}\right).$$
Если случайные величины нормально распределены, то их сумма нормально распределена.

Введём
$$\hat{U}_n = \frac{\hat{\sigma}_n^2}{\sigma^2} \cdot n.$$
Эта случайная величина независима с $\hat{a}_n, \hat{U}_n \sim \chi^2$ с $n-1$ степенями свободы.

Плотность $\chi^2$ распределения с $n$ степенями свободы:
$$
t_n \left( u \right) =
\frac{1}{2^{\frac{n}{2}} \cdot \Gamma \left( \frac{n}{2}\right)} \cdot u^{\frac{n}{2} - 1} \cdot e^{-\frac{u}{2}},
u \geq 0,
\Gamma \left( \alpha \right) = \int \limits_0^{\infty } u^{\alpha - 1} \cdot e^{-u} du.
$$

Используем первый пункт леммы 1, или
$$\hat{V}_n = \frac{\hat{a}_n - a}{\sigma} \cdot \sqrt{n} \sim N \left( 0, 1 \right).$$
Это означает, что $P \left\{ \left| \hat{V}_n \right| < z\right\} = 2 \Phi \left( z \right) = 1 - \gamma,$
где
$$\Phi \left( z \right) = \frac{1}{\sqrt{2\pi}} \int \limits_0^z e^{-\frac{u^2}{2}} du, z > 0.$$
Это функция Лапласа.

Из уравнения $2\Phi \left( z \right) = 1 - \gamma$ по таблицам находим $z_{\gamma}$.
Тогда $P \left\{ \left| \hat{V}_n \right| < z_{\gamma }\right\} = 1 - \gamma$.

Решаем уравнение относительно $a$.
Получим симметрический доверительный интервал
$$\hat{a}_n - \frac{z_{\gamma} \cdot \sigma}{\sqrt{n}} <
a <
\hat{a}_n + \frac{z_{\gamma} \cdot \sigma}{\sqrt{n}}.$$

Можем не попасть в этот интервал с вероятностью $\gamma$.

Абсолютная погрешность:
$$\frac{z_{\gamma} \cdot \sigma}{\sqrt{n}}.$$

In [3]:
def mean_interval_known_sigma(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    z = norm.ppf(1 - gamma / 2.)
    left = a_n - z * sigma / sqrt(n)
    right = a_n + z * sigma / sqrt(n)
    length = right - left
    print "a_n:", a_n
    print "mean 95%% confidence interval (theory), n =", n, (left, right)
    print "mean 95%% confidence interval (python), n =", n, _zconfint_generic(a_n, sigma / sqrt(n), gamma, 'two-sided')
    print "interval length:", length
    return length

In [4]:
l1 = mean_interval_known_sigma(100)

a_n: -0.0823398682889
mean 95%% confidence interval (theory), n = 100 (-0.27833626674285772, 0.11365653016515308)
mean 95%% confidence interval (python), n = 100 (-0.27833626674285772, 0.11365653016515308)
interval length: 0.391992796908


In [5]:
l2 = mean_interval_known_sigma(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

a_n: -0.0196104928622
mean 95%% confidence interval (theory), n = 1000 (-0.081589996092625988, 0.042369010368286331)
mean 95%% confidence interval (python), n = 1000 (-0.081589996092625988, 0.042369010368286324)
interval length: 0.123959006461
length ratio: 0.316227766017
1 / sqrt(n2 / n1): 0.316227766017


In [6]:
l3 = mean_interval_known_sigma(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n2):", 1 / sqrt(10)

a_n: -0.00168689435729
mean 95%% confidence interval (theory), n = 10000 (-0.02128653420269408, 0.017912745488107)
mean 95%% confidence interval (python), n = 10000 (-0.02128653420269408, 0.017912745488107)
interval length: 0.0391992796908
length ratio: 0.316227766017
1 / sqrt(n3 / n2): 0.316227766017


In [7]:
l4 = mean_interval_known_sigma(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

a_n: 0.0067853666538
mean 95%% confidence interval (theory), n = 100000 (0.00058741633075147948, 0.012983316976842709)
mean 95%% confidence interval (python), n = 100000 (0.00058741633075147948, 0.012983316976842709)
interval length: 0.0123959006461
length ratio: 0.316227766017
1 / sqrt(n4 / n3): 0.316227766017


При всех $n$ доверительный интервал накрывает $0$.
Скорость сходимости доверительного интервала к истинному значению математического ожидания $a = 0$ равна
$1 / \sqrt{n}$.
Результаты,
полученные с помощью теоретических выкладок и функции `_zconfint_generic(mean, std_mean, gamma, alternative)`
совпадают.

### б)
для математического ожидания $a$, когда дисперсия неизвестна.

**Лемма 2** Определим
\begin{equation}
\hat{V}_n = \frac{\hat{a}_n - a}{ \sigma} \cdot \sqrt{n}.
\end{equation}
Тогда
\begin{equation}
\hat{Z}_n = \sqrt{n - 1} \cdot \frac{\hat{V}_n}{\sqrt{\hat{U}_n}}
\end{equation}
имеет распределение Стьюдента с $n-1$ степенью свободы.
Плотность распределения:
\begin{equation}
S_n \left( u \right) =
\frac{\Gamma \left( \frac{n + 1}{2}\right)}{\sqrt{n \cdot \pi} \cdot \Gamma \left( \frac{n}{2}\right)} \cdot
\left( 1 + \frac{u^2}{n} \right)^{-\frac{n+1}{2}},
-\infty < u < +\infty.
\end{equation}

Используем лемму 2
\begin{equation}
\hat{Z}_n =
\sqrt{n-1} \cdot \frac{\sqrt{n} \cdot
\left( \hat{a}_n - a\right) \cdot \sigma}{\sigma \cdot \sqrt{n} \cdot \hat{\sigma}_n} =
\sqrt{n - 1} \cdot \frac{\hat{a}_n - a}{\hat{\sigma}_n}.
\end{equation}
Получается, что
\begin{equation}
P \left\{ \left| \hat{Z}_n\right| < z \right\} = 2 \int \limits_0^1 S_{n-1} \left( u \right) du = 1 - \gamma.
\end{equation}
Решаем:
\begin{equation}
\left| \hat{Z}_n\right| < z_{\gamma} \Rightarrow
\hat{a}_n - \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n-1}} <
a <
\hat{a}_n + \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n-1}}.
\end{equation}
Отличия:
- дисперсию $\sigma$ заменяем на оценку $\hat{\sigma}_n$;
- $\sqrt{n} \to \sqrt{n-1}$;
- $N \to S_{n-1}$, то есть $z_{gamma}$ вычисляется по другим таблицам.

In [8]:
def mean_interval_unknown_sigma(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    sigma_n = sqrt(((X - a_n) ** 2).mean())
    z = t.ppf(1 - gamma / 2., n - 1)
    left = a_n - z * sigma_n / sqrt(n - 1)
    right = a_n + z * sigma_n / sqrt(n - 1)
    length = right - left
    print "a_n:", a_n
    print "mean 95%% confidence interval, n =", n, (left, right)
    print "interval length:", length
    return length

In [9]:
l1 = mean_interval_unknown_sigma(100)

a_n: 0.0701596704772
mean 95%% confidence interval, n = 100 (-0.10044477998849252, 0.24076412094296576)
interval length: 0.341208900931


In [10]:
l2 = mean_interval_unknown_sigma(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

a_n: 0.0076217442764
mean 95%% confidence interval, n = 1000 (-0.053612036196105689, 0.068855524748896946)
interval length: 0.122467560945
length ratio: 0.358922526964
1 / sqrt(n2 / n1): 0.316227766017


In [11]:
l3 = mean_interval_unknown_sigma(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n2):", 1 / sqrt(10)

a_n: -0.00643849611523
mean 95%% confidence interval, n = 10000 (-0.026024769197245119, 0.013147776966782293)
interval length: 0.039172546164
length ratio: 0.319860588892
1 / sqrt(n3 / n2): 0.316227766017


In [12]:
l4 = mean_interval_unknown_sigma(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

a_n: -0.00149274677593
mean 95%% confidence interval, n = 100000 (-0.0076789020835774779, 0.0046934085317199663)
interval length: 0.0123723106153
length ratio: 0.315841369195
1 / sqrt(n4 / n3): 0.316227766017


Скорость сходимости доверительного интервала к истинному значению математического ожидания $a = 0$ равна
$\frac{1}{\sqrt{n}}$.
Доверительный интервал всегда накрывает $0$.
В данном случае теоретические результаты отличаются от результатов,
полученных с помощью функции `_tconfint_generic(mean, std_mean, dof, gamma, alternative)` отличаются,
так как в функции нет деления на $\sqrt{n-1}$.

### в)
для математического ожидания $a$, когда распределение не $N \left( 0, 1 \right)$, то есть мы не знаем,
что распределение нормальное.
Просто есть последовательность $\vec{X}$.

Считаем, что $M \xi = a, D \xi < \infty $.

Пользуемся центральной предельной теоремой.

Пусть несмещённая оценка дисперсии (выборочная несмещённая дисперсия)
\begin{equation}
\hat{\sigma}_n = \frac{1}{n - 1} \cdot \left( \sum \limits_{i = 1}^n X_i^2 - \frac{1}{n} \cdot \hat{a}_n^2 \right).
\end{equation}
Согласно центральной предельной теореме выходит
\begin{equation}
P \left\{ \sqrt{n} \cdot \frac{\left| \hat{a}_n - a \right|}{\sigma} < z \right\}
\overset{n \to \infty }{\to} \Phi \left( z \right) =
1 - \gamma.
\end{equation}
В этом случае будем строить ассимптотические доверительные интервалы
\begin{equation}
\hat{a}_n - \frac{z_{\gamma} \cdot \sigma}{\sqrt{n}} < a < \hat{a}_n + \frac{z_{\gamma} \cdot \sigma}{ \sqrt{n}}.
\end{equation}
Проблема в том, что $\sigma$ неизвестно.
На практике заменяют $\sigma$ на $\hat{\sigma}_n$, то есть доверительный интервал строится таким образом
\begin{equation}
\hat{a}_n = \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n}} <
a <
\hat{a}_n + \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n}},
\end{equation}
где $z_{\gamma}$ находится через функцию Лапласа.

In [13]:
def mean_interval_unknown_distribution(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    sigma_n = (sum(X ** 2) - mean(a_n ** 2)) / (n - 1)
    z = norm.ppf(1 - gamma)
    left = a_n - z * sigma_n / sqrt(n)
    right = a_n + z * sigma_n / sqrt(n)
    length = right - left
    print "a_n:", a_n
    print "mean 95%% confidence interval, n =", n, (left, right)
    print "interval length:", length
    return length

In [14]:
l1 = mean_interval_unknown_distribution(100)

a_n: 0.0045263701617
mean 95%% confidence interval, n = 100 (-0.15989674800485082, 0.16894948832825635)
interval length: 0.328846236333


In [15]:
l2 = mean_interval_unknown_distribution(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

a_n: 0.0417248701369
mean 95%% confidence interval, n = 1000 (-0.012414581514845785, 0.095864321788599216)
interval length: 0.108278903303
length ratio: 0.329269097043
1 / sqrt(n2 / n1): 0.316227766017


In [16]:
l3 = mean_interval_unknown_distribution(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n2):", 1 / sqrt(10)

a_n: -0.0106597280209
mean 95%% confidence interval, n = 10000 (-0.026637916983519729, 0.0053184609418096759)
interval length: 0.0319563779253
length ratio: 0.295130232671
1 / sqrt(n3 / n2): 0.316227766017


In [17]:
l4 = mean_interval_unknown_distribution(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

a_n: 0.00235355245384
mean 95%% confidence interval, n = 100000 (-0.0028284795296746866, 0.0075355844373640388)
interval length: 0.010364063967
length ratio: 0.32431910748
1 / sqrt(n4 / n3): 0.316227766017


Скорость сходимости доверительного интервала к истинному значению математического ожидания $a = 0$ равна
$\frac{1}{\sqrt{n}}$.
Доверительный интервал всегда накрывает $0$.

## Задание 2
При заданных $n$ строим доверительный интервал для дисперсии $\sigma^2 = 1$ (в допущении $N \left( 0, 1 \right)$).
Действительно ли он сужается и с какой скоростью?
Действительно ли этот интервал накрывает единицу?

Используем вторую часть первой леммы.

В данном случае дисперсия принимает значения только больше нуля, $\chi^2$ распределение не симметрично.
Оно имеет вид:
![chi-squared distribution](chi-squared.jpg)
Нужно строить такой доверительный интервал,
чтобы расстояние между точками $z_{\gamma}^{\left( 1 \right)}$ и $z_{\gamma}^{\left( 2 \right)}$ было маленьким:
\begin{equation}
\int \limits_0^{z_{\gamma}^{\left( 1 \right)}} t_{n-1} \left( u \right) du = \frac{\gamma}{2}, 
\int \limits_{z_{\gamma}^{\left( 2 \right)}} t_{n - 1} \left( u \right) du = \frac{\gamma}{2}.
\end{equation}
При заданном $\gamma$ из этих уравнений находим $z_{\gamma}^{\left( 1 \right) }$ и $z_{\gamma}^{\left( 2 \right)}$.
Получается,
что $P \left\{ z_{\gamma}^{\left( 1 \right) } < \hat{U}_n < z_{\gamma}^{\left( 2 \right) } \right\} = 1 - \gamma$.

Доверительный интервал для дисперсии
\begin{equation}
\frac{\left( n - 1 \right) \cdot \hat{\sigma}_n^2}{z_{\gamma n}^{\left( 2 \right)}} <
\sigma^2 <
\frac{\left( n - 1 \right) \cdot \hat{\sigma}_n^2}{z_{\gamma n}^{\left( 2 \right) }}.
\end{equation}
Когда увеличиваем $n$, увеличивабтся $z_{\gamma n}^{\left( i \right) }$.
Сам интервал сужается.

In [18]:
def variance_interval(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    sigma_n = (sum(X ** 2) - mean(a_n ** 2)) / (n - 1)
    z1 = chi2.ppf(gamma / 2, n - 1)
    z2 = chi2.ppf(1 - gamma / 2, n - 1)
    left = (n - 1) * sigma_n / z2
    right = (n - 1) * sigma_n / z1
    length = right - left
    print "sigma_n:", sigma_n
    print "variance 95%% confidence interval, n =", n, (left, right)
    print "interval length:", length
    return length

In [19]:
l1 = variance_interval(100)

sigma_n: 0.893115989155
variance 95%% confidence interval, n = 100 (0.6884995619527432, 1.2052505592316722)
interval length: 0.516750997279


In [20]:
l2 = variance_interval(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

sigma_n: 1.00461609456
variance 95%% confidence interval, n = 1000 (0.92202425570619262, 1.0988835885776991)
interval length: 0.176859332872
length ratio: 0.342252523561
1 / sqrt(n2 / n1): 0.316227766017


In [21]:
l3 = variance_interval(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n1):", 1 / sqrt(10)

sigma_n: 0.993237787724
variance 95%% confidence interval, n = 10000 (0.96627069652029018, 1.0213552160192152)
interval length: 0.0550845194989
length ratio: 0.311459500636
1 / sqrt(n3 / n1): 0.316227766017


In [22]:
l4 = variance_interval(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

sigma_n: 1.00680912665
variance 95%% confidence interval, n = 100000 (0.99804212791463354, 1.0156926904281356)
interval length: 0.0176505625135
length ratio: 0.320426912571
1 / sqrt(n4 / n3): 0.316227766017


Интервал сужается к истинному значению дисперсии $\sigma^2 = 1$ со скоростью $\frac{1}{\sqrt{n}}$.
Этот интервал действительно покрывает единицу.

## Задание 3
Хотим вычислить величину
$I = P \left( \xi < \eta \right), \xi \sim F \left( u \right), \eta \sim G \left( u \right)$,
где $F \left( u \right)$ и $G \left( u \right) $ --- функции распределения.
$\xi \geq 0$ и $\eta > 0$ --- независимы.
Фиксируем $\xi$:
\begin{equation}
I = \int \limits_0^{\infty} F \left( u \right) dG \left( u \right) = MF \left( \eta \right).
\end{equation}
Фиксируем $\xi$:
\begin{equation}
I = \int \limits_0^{\infty } \left[ 1 - G \left( u \right) \right] dF \left( u \right) =
M \left[ 1 - G \left( \xi \right)\right].
\end{equation}
Если $\xi \gg \eta$ в среднем, то $\xi < \eta$ очень редко.

В первом случае дисперсия будет пропорциональна $p^2$.

Возьмём конкретные функции распределения: $F \left( u \right) = 1 - e^{- \left( \alpha \cdot u\right)^2}, u \geq 0$ --- распределение Вейбулла (Weibull), $G \left( u \right) = 1 - e^{-u^3}, u \geq 0$.

Чтобы сгенерировать $\xi$, нужно приравнять $1 - e^{-\left( \alpha \cdot u \right)^2}$ к $1 - \omega $,
чтобы найти $u$, то есть
\begin{equation}
\xi = -\frac{1}{\alpha} \cdot \left( \ln \omega \right)^{\frac{1}{2}} \sim F \left( u \right).
\end{equation}

$P \left\{ \xi < u\right\} =
P \left\{ F^{-1} \left( \omega \right) < u \right\} =
P \left\{ \omega < F \left( u \right) \right\} =
F \left( u \right),
\eta = - \left( \ln \omega \right)^{\frac{1}{3}}$.

Хотим вычислить интеграл методом Монте-Карло.
Уровень доверия возьмём $\gamma = 0.05$.

Относительную погрешность возьмём 1%, то есть $\varepsilon = 0.01$.

Для первого способа оценка интеграла
\begin{equation}
\hat{I}_n^{\left( 1 \right) } = \frac{1}{n} \sum \limits_{i=1}^n F \left( \eta_i \right),
\alpha = 0.1, 0.01, 0.001. 
\end{equation}
Вычислить $I$ точно.
Для каждого из способов найти $n$, когда относительная погрешность оценки --- 1%
\begin{equation}
n^* = \min n \geq n_0: n \geq \frac{z_{\gamma}^2 \cdot \hat{\sigma}_n^2}{\varepsilon^2 \cdot \hat{I}_n^2}.
\end{equation}
В первом случае $n_0 = 100$.

Во втором случае $n_0$ в зависимости от $\alpha$ нужно увеличивать, например, $n_0 = 1000, 10000, 100000$,
чтобы дисперсия была устойчивой.

In [23]:
def find_n_1(n_0, alpha, epsilon=0.01):
    omega = uniform(size=n_0)
    eta = [(-log(x)) ** (1 / 3.) for x in omega]
    F = array([1 - exp(-(alpha * u) ** 2) for u in eta])
    I_n = mean(F)
    i = n_0
    z = norm.ppf(1 - gamma / 2.)
    while i < z ** 2 * (mean(F ** 2) - (mean(F)) ** 2) / (epsilon ** 2 * I_n ** 2):
        i += 1
        new_uniform = uniform(size=1)
        new_eta = (-log(new_uniform)) ** (1 / 3.)
        append(F, 1 - exp(-(alpha * new_eta) ** 2))
        I_n = mean(F)
    left = I_n - z * (mean(F ** 2) - (mean(F)) ** 2) / sqrt(i)
    right = I_n + z * (mean(F ** 2) - (mean(F)) ** 2) / sqrt(i)
    print "I_n:", I_n
    print "integral 1 95%% confidence interval:", (left, right)
    return i

In [24]:
n = find_n_1(100, 0.1)
print "alpha = 0.1, n:", n

I_n: 0.00850960221211
integral 1 95%% confidence interval: (0.008509151554522006, 0.0085100528697008414)
alpha = 0.1, n: 14879


In [25]:
n = find_n_1(100, 0.01)
print "alpha = 0.01, n:", n

I_n: 8.6965183749e-05
integral 1 95%% confidence interval: (8.6965131032753189e-05, 8.6965236465212122e-05)
alpha = 0.01, n: 18665


In [26]:
n = find_n_1(100, 0.001)
print "alpha = 0.001, n:", n

I_n: 1.06664385236e-06
integral 1 95%% confidence interval: (1.0666438454366391e-06, 1.0666438592742454e-06)
alpha = 0.001, n: 14208


In [27]:
def find_n_2(n_0, alpha, epsilon=0.01):
    omega = uniform(size=n_0)
    xi = [1 / alpha * (-log(x)) ** (0.5) for x in omega]
    G = array([1 - exp(-u ** 3) for u in xi])
    I_n = mean(1 - G)
    i = n_0
    z = norm.ppf(1 - gamma / 2.)
    while i < z ** 2 * (mean((1 - G) ** 2) - I_n ** 2) / (epsilon ** 2 * I_n ** 2):
        i += 1
        new_uniform = uniform(size=1)
        new_xi = 1 / alpha * (-log(new_uniform)) ** (0.5)
        append(G, 1 - exp(-new_xi ** 3))
        I_n = mean(1 - G)
    left = I_n - z * (mean((1 - G) ** 2) - (mean(1 - G)) ** 2) / sqrt(i)
    right = I_n + z * (mean((1 - G) ** 2) - (mean(1 - G)) ** 2) / sqrt(i)
    print "I_n:", I_n
    print "integral 2 95%% confidence interval:", (left, right)
    print "alpha:", alpha
    return i

In [28]:
n = find_n_2(n_0=1000, alpha=0.1)
print "n:", n

I_n: 0.00927049762922
integral 2 95%% confidence interval: (0.0092632607312158558, 0.0092777345272335877)
alpha: 0.1
n: 2723887


При $\alpha = 0.01$ нужный размер выборки примерно равен $227127000$, а при $\alpha=0.001$ --- $3841430000$.

Надём точное значение интеграла, представленного первым способом.

In [29]:
def integrand_1(u):
    return 3 * (1 - exp(-(0.1 * u) ** 2)) * u **2 * exp(-u ** 3)

In [30]:
print "The exact value of first integral:", quad(integrand_1, 0, inf)[0]

The exact value of first integral: 0.00896825263134


Найдём точное значение интеграла, представленного вторым способом.

In [31]:
def integrand_2(u):
    return 2 * 0.1 ** 2 * u * exp(-(0.1 * u) ** 2 - u ** 3)

In [32]:
print "The exact value of second integral:", quad(integrand_2, 0, inf)[0]

The exact value of second integral: 0.00896825263134


Значения интегралов попадают в построенные доверительные интервалы.
Для вычисления первого интеграла с точностью 1% нужна выборка размера 15-20 тысяч наблюдений.
Для вычисления второго интеграла с точностью 1% выборка должна содержать больше миллиона элементов.